In [1]:
import os
import zipfile
from io import BytesIO
import torch
import numpy as np
import xarray as xr
import netCDF4 as nc
import pandas as pd
import simpy as sp
import logging

In [2]:
import cdsapi

In [3]:
os.chdir('..')

In [4]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets.utils import unzip_csv_or_nc_batch
from datasets.net_cfd import load_netcdf

# EDA And Feature Engineering ERA5 HH Harbor

In [6]:
# params
dataset = "reanalysis-era5-single-levels-timeseries"
center_lon, center_lat = 7.25, 53.5
start_date = '2025-03-01'
end_date = '2025-04-01'

In [ ]:
request = {
    "variable": [
        "2m_dewpoint_temperature",
        "mean_sea_level_pressure",
        "skin_temperature",
        "surface_pressure",
        "surface_solar_radiation_downwards",
        "sea_surface_temperature",
        "surface_thermal_radiation_downwards",
        "2m_temperature",
        "total_precipitation",
        "10m_u_component_of_wind",
        "10m_v_component_of_wind",
        "100m_u_component_of_wind",
        "100m_v_component_of_wind",
        "mean_wave_direction",
        "mean_wave_period",
        "significant_height_of_combined_wind_waves_and_swell"
    ],
    "location": {"longitude": center_lon, "latitude": center_lat},
    "date": [f"{start_date}/{end_date}"],
    "data_format": "netcdf"
}
client = cdsapi.Client()
client.retrieve(dataset, request).download('./test_data/test_b_compr.zip')

Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002099BDBFA10>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 1 of 500
Retrying in 120 seconds
Recovering from connection error [HTTPSConnectionPool(host='cds.climate.copernicus.eu', port=443): Max retries exceeded with url: /api/catalogue/v1/messages (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000209A49BD210>: Failed to resolve 'cds.climate.copernicus.eu' ([Errno 11001] getaddrinfo failed)"))], attempt 2 of 500
Retrying in 120 seconds


In [7]:
unzip_csv_or_nc_batch("test_data")

In [9]:
data = load_netcdf("./test_data/test_b.nc")
data

<xarray.Dataset> Size: 15kB
Dimensions:     (valid_time: 768)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 6kB 2025-03-01 ... 2025-04-01T23:...
    lat         float64 8B ...
    lon         float64 8B ...
Data variables:
    mwd         (valid_time) float32 3kB ...
    mwp         (valid_time) float32 3kB ...
    swh         (valid_time) float32 3kB ...
Attributes:
    Conventions:             CF-1.7
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_edition:            1
    GRIB_subCentre:          0
    history:                 2024-09-05T05:51 GRIB to CDM+CF via cfgrib-0.9.1...
    institution:             European Centre for Medium-Range Weather Forecasts

In [10]:
df = data.to_dataframe()

In [11]:
df

,mwd,mwp,swh,lat,lon
valid_time,,,,,
2025-03-01 00:00:00,342.125885,4.373734,0.486018,53.5,7.5
2025-03-01 01:00:00,341.137726,4.417136,0.474745,53.5,7.5
2025-03-01 02:00:00,340.209961,4.455212,0.463773,53.5,7.5
2025-03-01 03:00:00,339.432281,4.481649,0.453880,53.5,7.5
2025-03-01 04:00:00,338.790253,4.500447,0.445031,53.5,7.5
...,...,...,...,...,...
2025-04-01 19:00:00,9.985579,3.915524,0.471827,53.5,7.5
2025-04-01 20:00:00,13.635413,3.837394,0.479441,53.5,7.5
2025-04-01 21:00:00,16.802130,3.790474,0.481889,53.5,7.5
